In [61]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels as sm
from statsmodels.regression.linear_model import OLS


In [62]:
househould_debt = pd.read_csv(r"C:\Users\10\Downloads\TDSP.csv")
intrest_rates = pd.read_csv(r"C:\Users\10\Downloads\IRLTLT01USQ156N.csv")
unemployment_rate = pd.read_csv(r"C:\Users\10\Downloads\UNRATE.csv")
real_disposable_income = pd.read_csv(r"C:\Users\10\Downloads\DPIC96.csv") 
cpi = pd.read_csv(r"C:\Users\10\Downloads\CPALTT01USQ661S.csv")
Real_Effective_Exchange_Rates = pd.read_csv(r"C:\Users\10\Downloads\CCRETT01USQ661N.csv")
govt_trans = pd.read_csv(r"C:\Users\10\Downloads\W015RC1Q027SBEA.csv")
nominal_housing_wealth = pd.read_csv(r"C:\Users\10\Downloads\USSTHPI.csv")
financial_wealth_nom = pd.read_csv(r"C:\Users\10\Downloads\SPASTT01USQ657N.csv")

In [63]:
intrest_rates = intrest_rates.rename(columns={"IRLTLT01USQ156N": "intrest_rate(%)"})
househould_debt = househould_debt.rename(columns={'TDSP':"(%) of income"})
unemployment_rate = unemployment_rate.rename(columns={"UNRATE":"(%) of labor force"})
real_disposable_income = real_disposable_income.rename(columns={"DPIC96":"in billions of chained 2017 dollars"}) 
cpi = cpi.rename(columns={"CPALTT01USQ661S":"CPI(2015=100)"})  
Real_Effective_Exchange_Rates = Real_Effective_Exchange_Rates.rename(columns={"CCRETT01USQ661N":"REER_CPI(2015=100)"})   
govt_trans = govt_trans.rename(columns={"W015RC1Q027SBEA":"in billions of dollars"})
nominal_housing_wealth = nominal_housing_wealth.rename(columns={"USSTHPI":"Index 1980:Q1=100"})
financial_wealth_nom = financial_wealth_nom.rename(columns={"SPASTT01USQ657N":"growth rate"})    

In [64]:
financial_wealth_nom.head()

,observation_date,growth rate
0,1995-01-01,3.422487
1,1995-04-01,8.113217
2,1995-07-01,7.440827
3,1995-10-01,5.350784
4,1996-01-01,6.807556


In [ ]:
# ==================== SECTION 4: DATA PROCESSING ====================
print("\n" + "="*50)
print("STEP 4: DATA PROCESSING CODE")
print("="*50)

def process_consumption_data(data_file='filled_data.csv'):
    """
    Process the filled data template for regression analysis
    """
    # Load data
    df = pd.read_csv(data_file, index_col=0, parse_dates=True)
    
    # Calculate real values using CPI
    if 'cpi_index' in df.columns:
        base_cpi = df['cpi_index'].iloc[df['cpi_index'].first_valid_index()]
        df['cpi_normalized'] = (df['cpi_index'] / base_cpi) * 100
        
        # Convert nominal to real
        df['consumption_real'] = df['household_consumption_nominal'] / df['cpi_normalized'] * 100
        df['income_real'] = df['household_income'] / df['cpi_normalized'] * 100
        df['housing_wealth_real'] = df['housing_wealth_index'] / df['cpi_normalized'] * 100
        df['financial_wealth_real'] = df['stock_market_index'] / df['cpi_normalized'] * 100
    
    # Calculate inflation (year-on-year)
    df['inflation_yoy'] = df['cpi_index'].pct_change(periods=4) * 100
    
    # Expected inflation (4-quarter moving average)
    df['expected_inflation'] = df['inflation_yoy'].rolling(window=4).mean()
    
    # Real interest rate
    df['real_interest_rate'] = df['interest_rate'] - df['expected_inflation']
    
    # Exchange rate changes
    df['exchange_rate_change'] = df['exchange_rate'].pct_change() * 100
    
    # Income volatility (rolling standard deviation)
    df['income_volatility'] = df['income_real'].pct_change().rolling(window=8).std()
    
    # Take logs for regression
    log_vars = ['consumption_real', 'income_real', 'housing_wealth_real', 'financial_wealth_real']
    for var in log_vars:
        if var in df.columns:
            df[f'log_{var}'] = np.log(df[var].replace(0, np.nan))
    
    # Create lagged consumption
    df['log_consumption_lagged'] = df['log_consumption_real'].shift(1)
    
    return df

<h1>Reference for data </h1> <br>

cpi = Organization for Economic Co-operation and Development, Consumer Price Indices (CPIs, HICPs), COICOP 1999: Consumer Price Index: Total for United States [USACPIALLQINMEI], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/USACPIALLQINMEI, September 29, 2025 <br>

nominal_housing_wealth = U.S. Federal Housing Finance Agency, All-Transactions House Price Index for the United States [USSTHPI], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/USSTHPI, September 29, 2025. <br>

unemployment_rate = U.S. Bureau of Labor Statistics, Unemployment Rate [UNRATE], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/UNRATE, September 29, 2025.<br>

govt_sp = U.S. Bureau of Economic Analysis, Federal government current transfer payments: Government social benefits [W015RC1Q027SBEA], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/W015RC1Q027SBEA, September 29, 2025.

real household consumption = International Monetary Fund, Real Households Final Consumption Expenditure for United States [NCPHIRSAXDCUSQ], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/NCPHIRSAXDCUSQ, September 29, 2025 <br>

financial_wealth_nominal = Organization for Economic Co-operation and Development, Share Prices: All Shares/Broad: Total for United States [SPASTT01USQ657N], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/SPASTT01USQ657N, September 29, 2025.

In [65]:
# First, let's check the structure of one dataframe to see the date column
print("Sample dataframe structure:")
print(f"househould_debt columns: {househould_debt.columns.tolist()}")
print(f"househould_debt shape: {househould_debt.shape}")
print("\nFirst few rows:")
print(househould_debt.head())

Sample dataframe structure:
househould_debt columns: ['observation_date', '(%) of income']
househould_debt shape: (182, 2)

First few rows:
  observation_date  (%) of income
0       1980-01-01      10.608892
1       1980-04-01      10.629809
2       1980-07-01      10.402635
3       1980-10-01      10.253121
4       1981-01-01      10.291887


In [66]:
# Create a list of all dataframes with their names for easier processing
dataframes_dict = {
    'household_debt': househould_debt,
    'interest_rates': intrest_rates,
    'unemployment_rate': unemployment_rate,
    'real_disposable_income': real_disposable_income,
    'cpi': cpi,
    'real_effective_exchange_rates': Real_Effective_Exchange_Rates,
    'govt_transfers': govt_trans,
    'nominal_housing_wealth': nominal_housing_wealth,
    'financial_wealth_nominal': financial_wealth_nom
}

# Function to prepare each dataframe for merging
def prepare_dataframe_for_merge(df, df_name):
    """
    Set observation_date as index and ensure it's in datetime format
    """
    df_copy = df.copy()
    
    # Convert observation_date to datetime if it's not already
    df_copy['observation_date'] = pd.to_datetime(df_copy['observation_date'])
    
    # Set observation_date as index
    df_copy = df_copy.set_index('observation_date')
    
    print(f"{df_name}: {df_copy.shape[0]} observations from {df_copy.index.min()} to {df_copy.index.max()}")
    
    return df_copy

# Prepare all dataframes
prepared_dfs = {}
print("Preparing dataframes for merging:")
print("-" * 50)

for name, df in dataframes_dict.items():
    prepared_dfs[name] = prepare_dataframe_for_merge(df, name)

Preparing dataframes for merging:
--------------------------------------------------
household_debt: 182 observations from 1980-01-01 00:00:00 to 2025-04-01 00:00:00
interest_rates: 121 observations from 1995-01-01 00:00:00 to 2025-01-01 00:00:00
unemployment_rate: 361 observations from 1995-01-01 00:00:00 to 2025-01-01 00:00:00
real_disposable_income: 121 observations from 1995-01-01 00:00:00 to 2025-01-01 00:00:00
cpi: 281 observations from 1955-01-01 00:00:00 to 2025-01-01 00:00:00
real_effective_exchange_rates: 121 observations from 1995-01-01 00:00:00 to 2025-01-01 00:00:00
govt_transfers: 121 observations from 1995-01-01 00:00:00 to 2025-01-01 00:00:00
nominal_housing_wealth: 121 observations from 1995-01-01 00:00:00 to 2025-01-01 00:00:00
financial_wealth_nominal: 121 observations from 1995-01-01 00:00:00 to 2025-01-01 00:00:00


In [67]:
# Merge all dataframes using outer join (keeps all dates from all dataframes)
print("\nMerging all dataframes...")
print("-" * 50)

# Start with the first dataframe
merged_df = list(prepared_dfs.values())[0].copy()
merged_df_name = list(prepared_dfs.keys())[0]

print(f"Starting with {merged_df_name}: {merged_df.shape}")

# Merge each subsequent dataframe
for name, df in list(prepared_dfs.items())[1:]:
    print(f"Merging {name}: {df.shape}")
    merged_df = merged_df.merge(df, left_index=True, right_index=True, how='outer')
    print(f"After merging {name}: {merged_df.shape}")

print(f"\nFinal merged dataframe shape: {merged_df.shape}")
print(f"Date range: {merged_df.index.min()} to {merged_df.index.max()}")
print(f"Columns: {merged_df.columns.tolist()}")


Merging all dataframes...
--------------------------------------------------
Starting with household_debt: (182, 1)
Merging interest_rates: (121, 1)
After merging interest_rates: (182, 2)
Merging unemployment_rate: (361, 1)
After merging unemployment_rate: (422, 3)
Merging real_disposable_income: (121, 1)
After merging real_disposable_income: (422, 4)
Merging cpi: (281, 1)
After merging cpi: (522, 5)
Merging real_effective_exchange_rates: (121, 1)
After merging real_effective_exchange_rates: (522, 6)
Merging govt_transfers: (121, 1)
After merging govt_transfers: (522, 7)
Merging nominal_housing_wealth: (121, 1)
After merging nominal_housing_wealth: (522, 8)
Merging financial_wealth_nominal: (121, 1)
After merging financial_wealth_nominal: (522, 9)

Final merged dataframe shape: (522, 9)
Date range: 1955-01-01 00:00:00 to 2025-04-01 00:00:00
Columns: ['(%) of income', 'intrest_rate(%)', '(%) of labor force', 'in billions of chained 2017 dollars', 'CPI(2015=100)', 'REER_CPI(2015=100)', 

In [68]:
# Display information about the merged dataframe
print("\n" + "="*60)
print("MERGED DATAFRAME SUMMARY")
print("="*60)

print(f"\nDataframe Info:")
print(f"Shape: {merged_df.shape}")
print(f"Index (Date range): {merged_df.index.min()} to {merged_df.index.max()}")
print(f"Index name: {merged_df.index.name}")

print(f"\nColumns and their data types:")
for col in merged_df.columns:
    print(f"  - {col}: {merged_df[col].dtype}")

print(f"\nFirst 10 rows of merged dataframe:")
print(merged_df.head(10))

print(f"\nLast 10 rows of merged dataframe:")
print(merged_df.tail(10))


MERGED DATAFRAME SUMMARY

Dataframe Info:
Shape: (522, 9)
Index (Date range): 1955-01-01 00:00:00 to 2025-04-01 00:00:00
Index name: observation_date

Columns and their data types:
  - (%) of income: float64
  - intrest_rate(%): float64
  - (%) of labor force: float64
  - in billions of chained 2017 dollars: float64
  - CPI(2015=100): float64
  - REER_CPI(2015=100): float64
  - in billions of dollars: float64
  - Index 1980:Q1=100: float64
  - growth rate: float64

First 10 rows of merged dataframe:
                  (%) of income  intrest_rate(%)  (%) of labor force  \
observation_date                                                       
1955-01-01                  NaN              NaN                 NaN   
1955-04-01                  NaN              NaN                 NaN   
1955-07-01                  NaN              NaN                 NaN   
1955-10-01                  NaN              NaN                 NaN   
1956-01-01                  NaN              NaN              

In [69]:
# Check for missing values and data quality
print("\n" + "="*60)
print("DATA QUALITY ANALYSIS")
print("="*60)

print("\nMissing values per column:")
missing_info = merged_df.isnull().sum()
missing_pct = (missing_info / len(merged_df)) * 100

for col in merged_df.columns:
    print(f"  {col}: {missing_info[col]} missing ({missing_pct[col]:.1f}%)")

print(f"\nData availability by time period:")
print(f"Total observations: {len(merged_df)}")
print(f"Complete cases (no missing values): {merged_df.dropna().shape[0]}")

# Show data availability for common analysis periods
common_periods = [
    ('1995-2024', '1995-01-01', '2024-12-31'),
    ('2000-2024', '2000-01-01', '2024-12-31'),
    ('1980-2024', '1980-01-01', '2024-12-31')
]

for period_name, start_date, end_date in common_periods:
    period_data = merged_df.loc[start_date:end_date]
    complete_cases = period_data.dropna().shape[0]
    total_cases = period_data.shape[0]
    print(f"  {period_name}: {complete_cases}/{total_cases} complete cases ({(complete_cases/total_cases)*100:.1f}%)")

# Create a more descriptive column mapping
column_mapping = {
    '(%) of income': 'household_debt_pct_income',
    'intrest_rate(%)': 'interest_rate_pct',
    '(%) of labor force': 'unemployment_rate_pct',
    'in billions of chained 2017 dollars': 'real_disposable_income_billions',
    'CPI(2015=100)': 'cpi_index_2015_100',
    'REER_CPI(2015=100)': 'real_exchange_rate_2015_100',
    'in billions of dollars': 'govt_transfers_billions',
    'Index 1980:Q1=100': 'housing_price_index_1980_100',
    'growth rate': 'financial_wealth_growth_rate'
}

# Rename columns for better readability
merged_df_clean = merged_df.rename(columns=column_mapping)
print(f"\nRenamed columns for clarity:")
for old_name, new_name in column_mapping.items():
    print(f"  '{old_name}' → '{new_name}'")


DATA QUALITY ANALYSIS

Missing values per column:
  (%) of income: 340 missing (65.1%)
  intrest_rate(%): 401 missing (76.8%)
  (%) of labor force: 161 missing (30.8%)
  in billions of chained 2017 dollars: 401 missing (76.8%)
  CPI(2015=100): 241 missing (46.2%)
  REER_CPI(2015=100): 401 missing (76.8%)
  in billions of dollars: 401 missing (76.8%)
  Index 1980:Q1=100: 401 missing (76.8%)
  growth rate: 401 missing (76.8%)

Data availability by time period:
Total observations: 522
Complete cases (no missing values): 121
  1995-2024: 120/360 complete cases (33.3%)
  2000-2024: 100/300 complete cases (33.3%)
  1980-2024: 120/420 complete cases (28.6%)

Renamed columns for clarity:
  '(%) of income' → 'household_debt_pct_income'
  'intrest_rate(%)' → 'interest_rate_pct'
  '(%) of labor force' → 'unemployment_rate_pct'
  'in billions of chained 2017 dollars' → 'real_disposable_income_billions'
  'CPI(2015=100)' → 'cpi_index_2015_100'
  'REER_CPI(2015=100)' → 'real_exchange_rate_2015_100'

In [70]:
# Final merged dataframe with clean column names
print("="*80)
print("FINAL MERGED DATAFRAME")
print("="*80)

print(f"Shape: {merged_df_clean.shape}")
print(f"Index: observation_date (from {merged_df_clean.index.min()} to {merged_df_clean.index.max()})")
print(f"\nColumns:")
for i, col in enumerate(merged_df_clean.columns, 1):
    print(f"  {i}. {col}")

print(f"\nSample of the merged dataframe (first 5 rows):")
print(merged_df_clean.head())

print(f"\nTo access this merged dataframe, use: merged_df_clean")
print(f"To save to CSV: merged_df_clean.to_csv('merged_economic_data.csv')")

FINAL MERGED DATAFRAME
Shape: (522, 9)
Index: observation_date (from 1955-01-01 00:00:00 to 2025-04-01 00:00:00)

Columns:
  1. household_debt_pct_income
  2. interest_rate_pct
  3. unemployment_rate_pct
  4. real_disposable_income_billions
  5. cpi_index_2015_100
  6. real_exchange_rate_2015_100
  7. govt_transfers_billions
  8. housing_price_index_1980_100
  9. financial_wealth_growth_rate

Sample of the merged dataframe (first 5 rows):
                  household_debt_pct_income  interest_rate_pct  \
observation_date                                                 
1955-01-01                              NaN                NaN   
1955-04-01                              NaN                NaN   
1955-07-01                              NaN                NaN   
1955-10-01                              NaN                NaN   
1956-01-01                              NaN                NaN   

                  unemployment_rate_pct  real_disposable_income_billions  \
observation_date  

In [73]:
merged_df.info()
merged_df_clean.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 522 entries, 1955-01-01 to 2025-04-01
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   (%) of income                        182 non-null    float64
 1   intrest_rate(%)                      121 non-null    float64
 2   (%) of labor force                   361 non-null    float64
 3   in billions of chained 2017 dollars  121 non-null    float64
 4   CPI(2015=100)                        281 non-null    float64
 5   REER_CPI(2015=100)                   121 non-null    float64
 6   in billions of dollars               121 non-null    float64
 7   Index 1980:Q1=100                    121 non-null    float64
 8   growth rate                          121 non-null    float64
dtypes: float64(9)
memory usage: 56.9 KB


,household_debt_pct_income,interest_rate_pct,unemployment_rate_pct,real_disposable_income_billions,cpi_index_2015_100,real_exchange_rate_2015_100,govt_transfers_billions,housing_price_index_1980_100,financial_wealth_growth_rate
observation_date,,,,,,,,,
1955-01-01,NaN,NaN,NaN,NaN,11.30439,NaN,NaN,NaN,NaN
1955-04-01,NaN,NaN,NaN,NaN,11.28892,NaN,NaN,NaN,NaN
1955-07-01,NaN,NaN,NaN,NaN,11.29736,NaN,NaN,NaN,NaN
1955-10-01,NaN,NaN,NaN,NaN,11.33111,NaN,NaN,NaN,NaN
1956-01-01,NaN,NaN,NaN,NaN,11.33252,NaN,NaN,NaN,NaN


In [75]:
# Check the frequency of each dataset to identify monthly vs quarterly data
print("="*80)
print("ANALYZING DATA FREQUENCIES")
print("="*80)

dataframes_dict = {
    'household_debt': househould_debt,
    'interest_rates': intrest_rates,
    'unemployment_rate': unemployment_rate,
    'real_disposable_income': real_disposable_income,
    'cpi': cpi,
    'real_effective_exchange_rates': Real_Effective_Exchange_Rates,
    'govt_transfers': govt_trans,
    'nominal_housing_wealth': nominal_housing_wealth,
    'financial_wealth_nominal': financial_wealth_nom
}

# Function to determine data frequency
def analyze_frequency(df, name):
    """Analyze the frequency of the data"""
    df_copy = df.copy()
    df_copy['observation_date'] = pd.to_datetime(df_copy['observation_date'])
    df_copy = df_copy.sort_values('observation_date')
    
    # Calculate differences between consecutive dates
    date_diffs = df_copy['observation_date'].diff().dropna()
    
    # Get the most common difference
    most_common_diff = date_diffs.mode()
    
    # Sample of recent dates
    recent_dates = df_copy['observation_date'].tail(10).tolist()
    
    if len(most_common_diff) > 0:
        days = most_common_diff.iloc[0].days
        if days <= 35:  # Around 30 days = monthly
            frequency = "MONTHLY"
        elif days <= 100:  # Around 90 days = quarterly  
            frequency = "QUARTERLY"
        else:
            frequency = "ANNUAL or OTHER"
    else:
        frequency = "UNKNOWN"
    
    return frequency, recent_dates[:5]

print("Dataset Analysis:")
print("-" * 50)
for name, df in dataframes_dict.items():
    freq, sample_dates = analyze_frequency(df, name)
    print(f"{name:30} | {freq:10} | Sample dates: {sample_dates[0]} to {sample_dates[-1]}")
    print(f"{'':31} | {'':10} | Total observations: {len(df)}")
    print()

ANALYZING DATA FREQUENCIES
Dataset Analysis:
--------------------------------------------------
household_debt                 | QUARTERLY  | Sample dates: 2023-01-01 00:00:00 to 2024-01-01 00:00:00
                                |            | Total observations: 182

interest_rates                 | QUARTERLY  | Sample dates: 2022-10-01 00:00:00 to 2023-10-01 00:00:00
                                |            | Total observations: 121

unemployment_rate              | MONTHLY    | Sample dates: 2024-04-01 00:00:00 to 2024-08-01 00:00:00
                                |            | Total observations: 361

real_disposable_income         | QUARTERLY  | Sample dates: 2022-10-01 00:00:00 to 2023-10-01 00:00:00
                                |            | Total observations: 121

cpi                            | QUARTERLY  | Sample dates: 2022-10-01 00:00:00 to 2023-10-01 00:00:00
                                |            | Total observations: 281

real_effective_exchange_rates

In [76]:
# Function to convert monthly data to quarterly
def convert_monthly_to_quarterly(df, value_column, aggregation_method='mean'):
    """
    Convert monthly data to quarterly data
    
    Parameters:
    df: DataFrame with 'observation_date' column
    value_column: Name of the column to aggregate
    aggregation_method: 'mean', 'sum', 'last' (end of quarter value)
    """
    df_copy = df.copy()
    df_copy['observation_date'] = pd.to_datetime(df_copy['observation_date'])
    df_copy = df_copy.set_index('observation_date')
    
    # Create quarterly periods
    df_copy['quarter'] = df_copy.index.to_period('Q')
    
    # Aggregate by quarter
    if aggregation_method == 'mean':
        quarterly_data = df_copy.groupby('quarter')[value_column].mean()
    elif aggregation_method == 'sum':
        quarterly_data = df_copy.groupby('quarter')[value_column].sum()
    elif aggregation_method == 'last':
        quarterly_data = df_copy.groupby('quarter')[value_column].last()
    else:
        quarterly_data = df_copy.groupby('quarter')[value_column].mean()
    
    # Convert period index back to timestamp (start of quarter)
    quarterly_data.index = quarterly_data.index.to_timestamp()
    
    return quarterly_data.to_frame()

# Function to ensure quarterly frequency for all datasets
def prepare_quarterly_data(df, name, value_column):
    """Prepare data ensuring quarterly frequency"""
    df_copy = df.copy()
    df_copy['observation_date'] = pd.to_datetime(df_copy['observation_date'])
    
    # Determine frequency
    freq, _ = analyze_frequency(df_copy, name)
    
    if freq == "MONTHLY":
        print(f"Converting {name} from MONTHLY to QUARTERLY using mean aggregation")
        quarterly_df = convert_monthly_to_quarterly(df_copy, value_column, 'mean')
    else:
        print(f"{name} is already {freq} - keeping as is")
        quarterly_df = df_copy.set_index('observation_date')[[value_column]]
    
    return quarterly_df

print("="*80)
print("CONVERTING TO QUARTERLY FREQUENCY (Q1 1995 - Q4 2024)")
print("="*80)

# Define the columns for each dataset
dataset_columns = {
    'household_debt': '(%) of income',
    'interest_rates': 'intrest_rate(%)',
    'unemployment_rate': '(%) of labor force', 
    'real_disposable_income': 'in billions of chained 2017 dollars',
    'cpi': 'CPI(2015=100)',
    'real_effective_exchange_rates': 'REER_CPI(2015=100)',
    'govt_transfers': 'in billions of dollars',
    'nominal_housing_wealth': 'Index 1980:Q1=100',
    'financial_wealth_nominal': 'growth rate'
}

# Prepare all datasets as quarterly
quarterly_datasets = {}
print("\nProcessing each dataset:")
print("-" * 50)

for name, df in dataframes_dict.items():
    value_col = dataset_columns[name]
    quarterly_datasets[name] = prepare_quarterly_data(df, name, value_col)

CONVERTING TO QUARTERLY FREQUENCY (Q1 1995 - Q4 2024)

Processing each dataset:
--------------------------------------------------
household_debt is already QUARTERLY - keeping as is
interest_rates is already QUARTERLY - keeping as is
Converting unemployment_rate from MONTHLY to QUARTERLY using mean aggregation
real_disposable_income is already QUARTERLY - keeping as is
cpi is already QUARTERLY - keeping as is
real_effective_exchange_rates is already QUARTERLY - keeping as is
govt_transfers is already QUARTERLY - keeping as is
nominal_housing_wealth is already QUARTERLY - keeping as is
financial_wealth_nominal is already QUARTERLY - keeping as is


In [77]:
# Merge all quarterly datasets
print("\n" + "="*60)
print("MERGING QUARTERLY DATASETS")
print("="*60)

# Start with the first dataset
merged_quarterly = list(quarterly_datasets.values())[0].copy()
dataset_names = list(quarterly_datasets.keys())

print(f"Starting with {dataset_names[0]}: {merged_quarterly.shape}")

# Merge each subsequent dataset
for i, (name, df) in enumerate(list(quarterly_datasets.items())[1:], 1):
    print(f"Merging {name}: {df.shape}")
    merged_quarterly = merged_quarterly.merge(df, left_index=True, right_index=True, how='outer')
    print(f"After merging {name}: {merged_quarterly.shape}")

print(f"\nMerged quarterly data shape: {merged_quarterly.shape}")
print(f"Date range: {merged_quarterly.index.min()} to {merged_quarterly.index.max()}")

# Filter to Q1 1995 - Q4 2024
print(f"\nFiltering to Q1 1995 - Q4 2024 period...")
start_date = '1995-01-01'
end_date = '2024-12-31'

# Filter the data
quarterly_filtered = merged_quarterly.loc[start_date:end_date].copy()

print(f"Filtered data shape: {quarterly_filtered.shape}")
print(f"Filtered date range: {quarterly_filtered.index.min()} to {quarterly_filtered.index.max()}")

# Create clean column names
clean_column_mapping = {
    '(%) of income': 'household_debt_pct_income',
    'intrest_rate(%)': 'interest_rate_pct',
    '(%) of labor force': 'unemployment_rate_pct',
    'in billions of chained 2017 dollars': 'real_disposable_income_billions',
    'CPI(2015=100)': 'cpi_index_2015_100',
    'REER_CPI(2015=100)': 'real_exchange_rate_2015_100',
    'in billions of dollars': 'govt_transfers_billions',
    'Index 1980:Q1=100': 'housing_price_index_1980_100',
    'growth rate': 'financial_wealth_growth_rate'
}

# Apply clean column names
quarterly_final = quarterly_filtered.rename(columns=clean_column_mapping)

print(f"\nFinal quarterly dataset:")
print(f"Shape: {quarterly_final.shape}")
print(f"Period: {quarterly_final.index.min()} to {quarterly_final.index.max()}")
print(f"Columns: {quarterly_final.columns.tolist()}")


MERGING QUARTERLY DATASETS
Starting with household_debt: (182, 1)
Merging interest_rates: (121, 1)
After merging interest_rates: (182, 2)
Merging unemployment_rate: (121, 1)
After merging unemployment_rate: (182, 3)
Merging real_disposable_income: (121, 1)
After merging real_disposable_income: (182, 4)
Merging cpi: (281, 1)
After merging cpi: (282, 5)
Merging real_effective_exchange_rates: (121, 1)
After merging real_effective_exchange_rates: (282, 6)
Merging govt_transfers: (121, 1)
After merging govt_transfers: (282, 7)
Merging nominal_housing_wealth: (121, 1)
After merging nominal_housing_wealth: (282, 8)
Merging financial_wealth_nominal: (121, 1)
After merging financial_wealth_nominal: (282, 9)

Merged quarterly data shape: (282, 9)
Date range: 1955-01-01 00:00:00 to 2025-04-01 00:00:00

Filtering to Q1 1995 - Q4 2024 period...
Filtered data shape: (120, 9)
Filtered date range: 1995-01-01 00:00:00 to 2024-10-01 00:00:00

Final quarterly dataset:
Shape: (120, 9)
Period: 1995-01-01 

In [78]:
# Final analysis of the quarterly dataset (Q1 1995 - Q4 2024)
print("\n" + "="*80)
print("FINAL QUARTERLY DATASET ANALYSIS (Q1 1995 - Q4 2024)")
print("="*80)

print(f"\nDataset Overview:")
print(f"Shape: {quarterly_final.shape}")
print(f"Period: Q1 1995 to Q4 2024 ({quarterly_final.shape[0]} quarters)")
print(f"Expected quarters for 30 years: {30 * 4} quarters")

print(f"\nIndex (observation_date):")
print(f"Start: {quarterly_final.index.min()}")
print(f"End: {quarterly_final.index.max()}")
print(f"Frequency: Quarterly")

print(f"\nMissing Values Analysis (NOT dropping nulls as requested):")
print("-" * 60)
missing_analysis = quarterly_final.isnull().sum()
missing_pct = (missing_analysis / len(quarterly_final)) * 100

for col in quarterly_final.columns:
    missing_count = missing_analysis[col]
    missing_percent = missing_pct[col]
    available_count = len(quarterly_final) - missing_count
    print(f"{col:35} | Missing: {missing_count:3d} ({missing_percent:5.1f}%) | Available: {available_count:3d}")

print(f"\nComplete cases (all variables available): {quarterly_final.dropna().shape[0]} quarters")

print(f"\nSample of quarterly data (first 10 quarters):")
print(quarterly_final.head(10))

print(f"\nSample of quarterly data (last 10 quarters):")
print(quarterly_final.tail(10))

print(f"\nTo access this dataset: quarterly_final")
print(f"To save: quarterly_final.to_csv('quarterly_economic_data_1995_2024.csv')")


FINAL QUARTERLY DATASET ANALYSIS (Q1 1995 - Q4 2024)

Dataset Overview:
Shape: (120, 9)
Period: Q1 1995 to Q4 2024 (120 quarters)
Expected quarters for 30 years: 120 quarters

Index (observation_date):
Start: 1995-01-01 00:00:00
End: 2024-10-01 00:00:00
Frequency: Quarterly

Missing Values Analysis (NOT dropping nulls as requested):
------------------------------------------------------------
household_debt_pct_income           | Missing:   0 (  0.0%) | Available: 120
interest_rate_pct                   | Missing:   0 (  0.0%) | Available: 120
unemployment_rate_pct               | Missing:   0 (  0.0%) | Available: 120
real_disposable_income_billions     | Missing:   0 (  0.0%) | Available: 120
cpi_index_2015_100                  | Missing:   0 (  0.0%) | Available: 120
real_exchange_rate_2015_100         | Missing:   0 (  0.0%) | Available: 120
govt_transfers_billions             | Missing:   0 (  0.0%) | Available: 120
housing_price_index_1980_100        | Missing:   0 (  0.0%) | Av